In [ ]:
import csv
import pandas as pd
df = pd.read_csv('20240512.csv', encoding='euc-kr')

df

,기준시,기준일자,AVC_ID,도로이정,이정,AVC차종구분명,AVC차종구분코드,교통량,콘존명,콘존길이,Unnamed: 10
0,0,2024-05-12,0010AV00260,40.11,53.40,10종,10,3,언양JC-활천IC,17.09,NaN
1,0,2024-05-12,0010AV00260,40.11,53.40,11종,11,0,언양JC-활천IC,17.09,NaN
2,0,2024-05-12,0010AV00260,40.11,53.40,12종,12,0,언양JC-활천IC,17.09,NaN
3,0,2024-05-12,0010AV00260,40.11,53.40,1종,1,134,언양JC-활천IC,17.09,NaN
4,0,2024-05-12,0010AV00260,40.11,53.40,2종,2,6,언양JC-활천IC,17.09,NaN
...,...,...,...,...,...,...,...,...,...,...,...
49291,23,2024-05-12,6000AV02600,27.96,26.69,6종,6,0,김해가야Hi-대감JC,1.77,NaN
49292,23,2024-05-12,6000AV02600,27.96,26.69,7종,7,0,김해가야Hi-대감JC,1.77,NaN
49293,23,2024-05-12,6000AV02600,27.96,26.69,8종,8,0,김해가야Hi-대감JC,1.77,NaN
49294,23,2024-05-12,6000AV02600,27.96,26.69,9종,9,0,김해가야Hi-대감JC,1.77,NaN


In [ ]:
#처리할 파일 선택
file_names = [f'202405/202405{str(day).zfill(2)}.csv' for day in range(1, 14)]


for file_name in file_names:
    # CSV 파일 읽기
    df = pd.read_csv(file_name, encoding='euc-kr')

    # 필요없는 컬럼, 미분류 코드 제거
    df = df.drop(columns=['Unnamed: 10'])
    df = df.drop(columns=['AVC차종구분명'])
    df = df[df['AVC차종구분코드'] != 13]

    # AVC차종구분코드가 1과 2인 행은 그대로 두고, 3에서 12인 행들의 'AVC차종구분코드' 값을 3으로 변경
    df.loc[(df['AVC차종구분코드'] >= 3) & (df['AVC차종구분코드'] <= 12), 'AVC차종구분코드'] = 3

    # 그룹화하여 교통량 값을 합치기
    df_grouped = df.groupby(['기준시', '기준일자', 'AVC_ID', '도로이정', '이정', 'AVC차종구분코드', '콘존명', '콘존길이']).agg({'교통량': 'sum'}).reset_index()

    # 컬럼명 수정
    new_column_names = {
        '기준시': 'time',
        '기준일자': 'date',
        'AVC_ID': 'AVC_ID',
        '도로이정': 'start',
        '이정': 'end',
        'AVC차종구분코드': 'car_type',
        '콘존명': 'name',
        '콘존길이': 'length',
        '교통량': 'traffic'
    }

    df_grouped.rename(columns=new_column_names, inplace=True)

    # 'car_type' 변경
    df_grouped.loc[df_grouped['car_type'] == 1, 'car_type'] = '승용'
    df_grouped.loc[df_grouped['car_type'] == 2, 'car_type'] = '버스'
    df_grouped.loc[df_grouped['car_type'] == 3, 'car_type'] = '화물'

    # 변경된 데이터프레임을 CSV 파일로 저장
    processed_file_name = f'processed_{file_name}'
    df_grouped.to_csv(processed_file_name, index=False, encoding='utf-8-sig')

    print(f'{file_name} 처리가 완료되었습니다.')



202405/20240501.csv 처리가 완료되었습니다.
202405/20240502.csv 처리가 완료되었습니다.
202405/20240503.csv 처리가 완료되었습니다.
202405/20240504.csv 처리가 완료되었습니다.
202405/20240505.csv 처리가 완료되었습니다.
202405/20240506.csv 처리가 완료되었습니다.
202405/20240507.csv 처리가 완료되었습니다.
202405/20240508.csv 처리가 완료되었습니다.
202405/20240509.csv 처리가 완료되었습니다.
202405/20240510.csv 처리가 완료되었습니다.
202405/20240511.csv 처리가 완료되었습니다.
202405/20240512.csv 처리가 완료되었습니다.
202405/20240513.csv 처리가 완료되었습니다.


In [ ]:
# 파일명 리스트 생성
file_names = [f'processed_202405/202405{str(day).zfill(2)}.csv' for day in range(1, 14)]

# 파일을 읽어 들여 데이터프레임으로 저장
dfs = []
for file_name in file_names:
    df = pd.read_csv(file_name, encoding='utf-8-sig')
    dfs.append(df)

# 데이터프레임을 하나로 합치기
combined_df = pd.concat(dfs, ignore_index=True)

# 합쳐진 데이터프레임을 하나의 CSV 파일로 저장
combined_df.to_csv('combined_data202405.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 파일명 리스트 생성
file_names = [f'combined_data202401.csv', f'combined_data202402.csv', f'combined_data202403.csv', f'combined_data202404.csv', f'combined_data202405.csv']

# 파일을 읽어 들여 데이터프레임으로 저장
dfs = []
for file_name in file_names:
    df = pd.read_csv(file_name, encoding='utf-8-sig')
    dfs.append(df)

# 데이터프레임을 하나로 합치기
combined_df = pd.concat(dfs, ignore_index=True)

# 합쳐진 데이터프레임을 하나의 CSV 파일로 저장 (UTF-8-SIG로 인코딩)
combined_df.to_csv('combined_data202401_202405.csv', index=False, encoding='utf-8-sig')

In [ ]:
import csv
import pandas as pd
df = pd.read_csv('combined_data202401_202405.csv', encoding='utf-8-sig')

df = df.drop(columns=['length'])

df

,time,date,AVC_ID,start,end,car_type,name,traffic
0,0,2024-01-01,0010AV00260,40.11,53.40,승용,언양JC-활천IC,174
1,0,2024-01-01,0010AV00260,40.11,53.40,버스,언양JC-활천IC,13
2,0,2024-01-01,0010AV00260,40.11,53.40,화물,언양JC-활천IC,22
3,0,2024-01-01,0010AV00260,57.20,53.40,승용,활천IC-언양JC,181
4,0,2024-01-01,0010AV00260,57.20,53.40,버스,활천IC-언양JC,4
...,...,...,...,...,...,...,...,...
1540903,23,2024-05-13,6000AV02600,26.19,26.69,버스,대감JC-김해가야Hi,0
1540904,23,2024-05-13,6000AV02600,26.19,26.69,화물,대감JC-김해가야Hi,0
1540905,23,2024-05-13,6000AV02600,27.96,26.69,승용,김해가야Hi-대감JC,0
1540906,23,2024-05-13,6000AV02600,27.96,26.69,버스,김해가야Hi-대감JC,0


In [ ]:
df['AVC_ID'].unique()

array(['경부선', '남해선(순천부산)', '0141AV00650 ', '서해안선', '울산선', '0201AV00300 ',
       '0201AV00500 ', '호남선', '0300AV01200 ', '당진대전선', '통영대전선/중부선', '중부선',
       '제2중부선', '평택제천선', '중부내륙선', '영동선', '중앙선', '대구외곽순환선', '수도권제1순환선',
       '남해제1지선', '제2경인선(인천안양)', '경인선', '서천공주선', '호남선의 지선', '대전남부순환선',
       '4005AVI3044 ', '4005AVI3050 ', '4510AV00070 ', '광주외곽순환선',
       '부산외곽순환선', '0321AVI2945 ', '0321AVI3207 ', '서울양양선'], dtype=object)

In [ ]:
filtered_df = df[df['AVC_ID'].str.startswith('1100')]

filtered_df
filtered_df['name'].unique()

array(['남인천TG-신천IC', '신천IC-남인천TG', '신천IC-안현JC', '안현JC-신천IC', '문학IC-학익JC',
       '문학IC-남동IC', '광명IC-안현JC', '광명IC-일직JC'], dtype=object)

In [ ]:
df.loc[df['AVC_ID'].str.startswith('0010'), 'AVC_ID'] = '경부선'
df.loc[df['AVC_ID'].str.startswith('0150'), 'AVC_ID'] = '서해안선'
df.loc[df['AVC_ID'].str.startswith('0160'), 'AVC_ID'] = '울산선'
df.loc[df['AVC_ID'].str.startswith('0251'), 'AVC_ID'] = '호남선'
df.loc[df['AVC_ID'].str.startswith('0301'), 'AVC_ID'] = '당진대전선'
df.loc[df['AVC_ID'].str.startswith('0351'), 'AVC_ID'] = '통영대전선/중부선'
df.loc[df['AVC_ID'].str.startswith('0352'), 'AVC_ID'] = '중부선'
df.loc[df['AVC_ID'].str.startswith('0370'), 'AVC_ID'] = '제2중부선'
df.loc[df['AVC_ID'].str.startswith('0400'), 'AVC_ID'] = '평택제천선'
df.loc[df['AVC_ID'].str.startswith('0450'), 'AVC_ID'] = '중부내륙선'
df.loc[df['AVC_ID'].str.startswith('0500'), 'AVC_ID'] = '영동선'
df.loc[df['AVC_ID'].str.startswith('0550'), 'AVC_ID'] = '중앙선'
df.loc[df['AVC_ID'].str.startswith('0600'), 'AVC_ID'] = '서울양양선'
df.loc[df['AVC_ID'].str.startswith('1000'), 'AVC_ID'] = '수도권제1순환선'
df.loc[df['AVC_ID'].str.startswith('1020'), 'AVC_ID'] = '남해제1지선'
df.loc[df['AVC_ID'].str.startswith('1200'), 'AVC_ID'] = '경인선'
df.loc[df['AVC_ID'].str.startswith('1510'), 'AVC_ID'] = '서천공주선'
df.loc[df['AVC_ID'].str.startswith('3000'), 'AVC_ID'] = '대전남부순환선'
df.loc[df['AVC_ID'].str.startswith('6000'), 'AVC_ID'] = '부산외곽순환선'
df.loc[df['AVC_ID'].str.startswith('0700'), 'AVC_ID'] = '대구외곽순환선'
df.loc[df['AVC_ID'].str.startswith('5000'), 'AVC_ID'] = '광주외곽순환선'
df.loc[df['AVC_ID'].str.startswith('0100'), 'AVC_ID'] = '남해선(순천부산)'
df.loc[df['AVC_ID'].str.startswith('2510'), 'AVC_ID'] = '호남선의 지선'
df.loc[df['AVC_ID'].str.startswith('1100'), 'AVC_ID'] = '제2경인선(인천안양)'

In [ ]:
df['direction'] = ''
df

,time,date,AVC_ID,start,end,car_type,name,traffic,direction
0,0,2024-01-01,경부선,40.11,53.40,승용,언양JC-활천IC,174,
1,0,2024-01-01,경부선,40.11,53.40,버스,언양JC-활천IC,13,
2,0,2024-01-01,경부선,40.11,53.40,화물,언양JC-활천IC,22,
3,0,2024-01-01,경부선,57.20,53.40,승용,활천IC-언양JC,181,
4,0,2024-01-01,경부선,57.20,53.40,버스,활천IC-언양JC,4,
...,...,...,...,...,...,...,...,...,...
1540903,23,2024-05-13,부산외곽순환선,26.19,26.69,버스,대감JC-김해가야Hi,0,
1540904,23,2024-05-13,부산외곽순환선,26.19,26.69,화물,대감JC-김해가야Hi,0,
1540905,23,2024-05-13,부산외곽순환선,27.96,26.69,승용,김해가야Hi-대감JC,0,
1540906,23,2024-05-13,부산외곽순환선,27.96,26.69,버스,김해가야Hi-대감JC,0,


In [ ]:
df.loc[df['start'] <= df['end'], 'direction'] = '상행선'
df.loc[df['start'] > df['end'], 'direction'] = '하행선'

In [ ]:
df

,time,date,AVC_ID,start,end,car_type,name,traffic,direction
0,0,2024-01-01,경부선,40.11,53.40,승용,언양JC-활천IC,174,상행선
1,0,2024-01-01,경부선,40.11,53.40,버스,언양JC-활천IC,13,상행선
2,0,2024-01-01,경부선,40.11,53.40,화물,언양JC-활천IC,22,상행선
3,0,2024-01-01,경부선,57.20,53.40,승용,활천IC-언양JC,181,하행선
4,0,2024-01-01,경부선,57.20,53.40,버스,활천IC-언양JC,4,하행선
...,...,...,...,...,...,...,...,...,...
1540903,23,2024-05-13,부산외곽순환선,26.19,26.69,버스,대감JC-김해가야Hi,0,상행선
1540904,23,2024-05-13,부산외곽순환선,26.19,26.69,화물,대감JC-김해가야Hi,0,상행선
1540905,23,2024-05-13,부산외곽순환선,27.96,26.69,승용,김해가야Hi-대감JC,0,하행선
1540906,23,2024-05-13,부산외곽순환선,27.96,26.69,버스,김해가야Hi-대감JC,0,하행선


In [ ]:
df['AVC_ID'].unique()

array(['경부선', '남해선(순천부산)', '서해안선', '울산선', '호남선', '당진대전선', '통영대전선/중부선',
       '중부선', '제2중부선', '평택제천선', '중부내륙선', '영동선', '중앙선', '대구외곽순환선',
       '수도권제1순환선', '남해제1지선', '제2경인선(인천안양)', '경인선', '서천공주선', '호남선의 지선',
       '대전남부순환선', '광주외곽순환선', '부산외곽순환선', '서울양양선'], dtype=object)

In [ ]:
# 'AVC_ID'가 숫자로 시작하는 행 제거
df = df[~df['AVC_ID'].str.match('^\d')]

In [ ]:
df

,time,date,AVC_ID,start,end,car_type,name,traffic,direction
0,0,2024-01-01,경부선,40.11,53.40,승용,언양JC-활천IC,174,상행선
1,0,2024-01-01,경부선,40.11,53.40,버스,언양JC-활천IC,13,상행선
2,0,2024-01-01,경부선,40.11,53.40,화물,언양JC-활천IC,22,상행선
3,0,2024-01-01,경부선,57.20,53.40,승용,활천IC-언양JC,181,하행선
4,0,2024-01-01,경부선,57.20,53.40,버스,활천IC-언양JC,4,하행선
...,...,...,...,...,...,...,...,...,...
1540903,23,2024-05-13,부산외곽순환선,26.19,26.69,버스,대감JC-김해가야Hi,0,상행선
1540904,23,2024-05-13,부산외곽순환선,26.19,26.69,화물,대감JC-김해가야Hi,0,상행선
1540905,23,2024-05-13,부산외곽순환선,27.96,26.69,승용,김해가야Hi-대감JC,0,하행선
1540906,23,2024-05-13,부산외곽순환선,27.96,26.69,버스,김해가야Hi-대감JC,0,하행선


In [ ]:
df = df.drop(columns=['start', 'end', 'name'])

In [ ]:
df

,time,date,AVC_ID,car_type,traffic,direction
0,0,2024-01-01,경부선,승용,174,상행선
1,0,2024-01-01,경부선,버스,13,상행선
2,0,2024-01-01,경부선,화물,22,상행선
3,0,2024-01-01,경부선,승용,181,하행선
4,0,2024-01-01,경부선,버스,4,하행선
...,...,...,...,...,...,...
1540903,23,2024-05-13,부산외곽순환선,버스,0,상행선
1540904,23,2024-05-13,부산외곽순환선,화물,0,상행선
1540905,23,2024-05-13,부산외곽순환선,승용,0,하행선
1540906,23,2024-05-13,부산외곽순환선,버스,0,하행선


In [ ]:
df_grouped = df.groupby(['time', 'date', 'AVC_ID', 'car_type', 'direction'])['traffic'].sum().reset_index()
df_grouped

,time,date,AVC_ID,car_type,direction,traffic
0,0,2024-01-01,경부선,버스,상행선,155
1,0,2024-01-01,경부선,버스,하행선,349
2,0,2024-01-01,경부선,승용,상행선,6784
3,0,2024-01-01,경부선,승용,하행선,6038
4,0,2024-01-01,경부선,화물,상행선,743
...,...,...,...,...,...,...
424180,23,2024-05-13,호남선의 지선,버스,하행선,9
424181,23,2024-05-13,호남선의 지선,승용,상행선,214
424182,23,2024-05-13,호남선의 지선,승용,하행선,466
424183,23,2024-05-13,호남선의 지선,화물,상행선,195


In [ ]:
df_grouped.to_csv('202401_202405_grouped.csv', encoding='utf-8-sig', index=False)

In [3]:
import csv
import pandas as pd
df = pd.read_csv('202401_202405_grouped.csv', encoding='utf-8-sig')

head = df.head(10)

head.to_csv('head_car_type.csv', encoding='utf-8-sig')

In [ ]:
# isnull() 또는 isna() 메서드를 사용하여 null 값을 찾음
null_values = df.isnull()

# null 값을 포함하는 행 또는 열을 찾고 싶다면 any() 메서드를 사용할 수 있음
# axis=0이면 열을, axis=1이면 행을 확인함
rows_with_null = df[df.isnull().any(axis=1)]
columns_with_null = df.columns[df.isnull().any()]

print("Null 값을 포함하는 행:")
print(rows_with_null)

print("\nNull 값을 포함하는 열:")
print(columns_with_null)

Null 값을 포함하는 행:
Empty DataFrame
Columns: [time, date, AVC_ID, car_type, traffic, direction]
Index: []

Null 값을 포함하는 열:
Index([], dtype='object')


In [ ]:
# 기준시가 0 이상 5 이하인 레코드들을 0으로 설정
df_grouped.loc[(df_grouped['기준시'] >= 0) & (df_grouped['기준시'] <= 5), '기준시'] = 0

# 기준시가 6 이상 11 이하인 레코드들을 6으로 설정
df_grouped.loc[(df_grouped['기준시'] >= 6) & (df_grouped['기준시'] <= 11), '기준시'] = 6

# 기준시가 12 이상 17 이하인 레코드들을 12으로 설정
df_grouped.loc[(df_grouped['기준시'] >= 12) & (df_grouped['기준시'] <= 17), '기준시'] = 12

# 기준시가 18 이상 23 이하인 레코드들을 18으로 설정
df_grouped.loc[(df_grouped['기준시'] >= 18) & (df_grouped['기준시'] <= 23), '기준시'] = 18

# 그룹화하여 교통량 값을 합치기
df_regrouped = df_grouped.groupby(['기준시', '기준일자', 'AVC_ID', '도로이정', '이정', 'AVC차종구분코드', '콘존명', '콘존길이']).agg({'교통량': 'sum'}).reset_index()